In [1]:
import pandas as pd
import numpy as np

# 自由现金流估值法 DCF

有以下四种模型：

1. 零增长模型
2. 不变增长模型
3. 两阶段模型
4. 三阶段模型

不同的是自由现金流的使用和贴现的方式不同。

**计算步骤**：

1. 计算自由现金流并依据相应的方法折现($\star\star\star\star\star$, the most important, this is what the code solves)
2. 计算股权价值= 1.+金融资产+长期股权投资-公司债务
3. 计算少数股东比例
4. 归属于上市公司股东的价值=股权价值$\times$(1-少数股东比例)
5. 每股内在价值=归属于上市公司股东的价值/股本

其中，

- 经营资产自由现金流=公司维持原有生产经营规模前提下的增量现金流入=经营活动现金流量净额-保全性资本支出=经营活动现金流量净额-固定资产折旧-无形资产和长期待摊费用摊销-处置长期资产的损失
- $WACC=k_d\times\frac{D}{D+E}\times(1-t)+k_e\times\frac{E}{D+E}$。其中债务资本成本率=债务资本总额/债务资本平均金额$\times$100%=(财务费用+汇兑收益)/(期初债务资本+期末债务资本)/2；股权资本成本率应该高于同期的国债利率，加上股票投资的风险溢价，我们普遍设置为8%；t为公司实际所得税税率=1-净利润/税前利润。
- 公司债务=有息债务
- 少数股东比例=$\frac{少数股东权益}{股东权益合计}$
- 股本=市值/股价


$$
\begin{aligned}
&零增长模型：V=\frac{FCF}{WACC}\\
&不变增长模型：V=\frac{FCF(1+g)}{WACC-g}\\
&两阶段模型：V=\sum_{t=1}^n\frac{{FCF}_t}{(1+WACC)^t}+\frac{TV}{(1+WACC)^n}，\ \  其中TV=\frac{FCF_n(1+g_2)}{WACC-g_2}\\
&三阶段模型：V=\sum_{t=1}^n\frac{{FCF}_0(1+g_1)}{(1+WACC)^t}+\sum_{t=n+1}^m\frac{{FCF}_n(1+g_2)}{(1+WACC)^t}+\frac{FCF_{n+m}(1+g_3)}{(WACC-g_3)(1+WACC)^{n+m}}\\
\end{aligned}
$$

零增长模型适用于成熟稳定、没有增长的公司，每年的自由现金流也保持在一个稳定的金额水平，类似于永续年金；如果该类公司的自由现金流全部用于发放股利现金，那么其得出的结果与股利贴现模型非常接近。

不变增长模型适用于成熟的公司，未来的自由现金流以非常缓慢的速度增长。

在两阶段模型中，投资者的预期回报WACC至少要高于总体的经济增长率；不变增长率g2通常小于WACC，反之，意味着很长时间以后公司的规模将超过总体经济规模。

在三阶段模型中，假设所有的公司经历三个阶段：成长阶段、过渡阶段和稳定阶段。三个阶段的成长率由高到低，稳定阶段保持较低增长率的不变增长。

In [40]:
#=== 变量
file_name = 'sz000977'  # 股票代码
time = 4  # 采用最近n期的数据(季)
zero_change = False  # 是否为零增长模型
one_change = False  # 是否为不变增长模型
two_change = True   # 是否为两阶段模型
three_change = False  # 是否为三阶段模型
g1, g2, g3 = 0.2, 0.03, 0.01  # 增长速度，分别是后三个模型的g，如果使用不变增长模型，则不需要更改后两个
t1, t2 = np.arange(1, 3), np.arange(1,2)  # 某阶段的几年，两阶段与三阶段模型需要，注意最大值减一为实际值

In [44]:
#=== functions
def read_file(file_name):
    # 读取股票基本数据
    df = pd.read_csv(r'youraddress\%s.csv' % file_name, encoding='GBK', skiprows=1, parse_dates=['交易日期'])
    df = df[['股票代码', '股票名称', '交易日期', '总市值', '净利润TTM', '收盘价']]
    print(df.tail(5))
    # 读取股票财务数据
    finance_df = pd.read_csv(r'youraddress\%s.csv' % file_name, parse_dates=['财报日期', '财报发布日期'], skiprows=1, encoding='gbk')
    finance_df = finance_df.resample('Q', on='财报日期').first()
    del finance_df['财报日期']
    finance_df.reset_index(inplace=True)
    finance_df.dropna(subset=['财报发布日期'], inplace=True)
    finance_df.sort_values(by='财报发布日期', inplace=True)
    return df, finance_df


def merge_data(df, finance_df):
    add_columns = ['B_货币资金',  
     'B_交易性金融资产',
     'B_衍生金融资产',
     'B_应收票据及应收账款',
     'B_应收票据',
     'B_应收账款',
     'B_应收款项融资',
     'B_应收利息',
     'B_应收股利',
     'B_其他应收款',
     'B_买入返售金融资产',
     'B_发放贷款及垫款',
     'B_可供出售金融资产',
     'B_持有至到期投资',
     'B_长期应收款',
     'B_长期股权投资',
     'B_投资性房地产',
     'B_所有者权益(或股东权益)合计',   
     'C_经营活动产生的现金流量净额', 
     'B_短期借款',
     'B_交易性金融负债',
     'B_应付利息',
     'B_应付短期债券',
     'B_一年内到期的非流动负债',
     'B_长期借款',
     'B_应付债券',
     'B_租赁负债',
     'B_长期应付款(合计)',
     'R_财务费用',
     'R_汇兑收益',
     'R_四、利润总额',
     'R_减：所得税费用',
                   'C_固定资产折旧、油气资产折耗、生产性物资折旧', 'C_无形资产摊销', 'C_长期待摊费用摊销', 'C_处置固定资产、无形资产和其他长期资产的损失',
                    'B_少数股东权益']
    col = ['财报发布日期', '财报日期'] + add_columns
    stock_df = pd.merge_asof(df, finance_df[col], left_on='交易日期', right_on='财报日期', direction='backward')
    print(stock_df.columns)
    
    return stock_df


def data_been_prepared(now_df, stock_df):
    now_df[['股票代码', '股票名称', '交易日期', '总市值', '财报发布日期', '财报日期', '净利润TTM', '收盘价']] = stock_df[['股票代码', '股票名称', '交易日期', '总市值', '财报发布日期', '财报日期', '净利润TTM', '收盘价']]
    now_df['金融资产'] = 0
    now_df['公司债务'] = 0
    for factor1 in ['B_货币资金',
     'B_交易性金融资产',
     'B_衍生金融资产',
     'B_应收票据及应收账款',
     'B_应收票据',
     'B_应收账款',
     'B_应收款项融资',
     'B_应收利息',
     'B_应收股利',
     'B_其他应收款',
     'B_买入返售金融资产',
     'B_发放贷款及垫款',
     'B_可供出售金融资产',
     'B_持有至到期投资',
     'B_长期应收款',
     'B_投资性房地产',
     'B_长期股权投资']:
        now_df['金融资产'] += stock_df[factor1]
    for factor2 in ['B_短期借款',
     'B_交易性金融负债',
     'B_应付利息',
     'B_应付短期债券',
     'B_一年内到期的非流动负债',
     'B_长期借款',
     'B_应付债券',
     'B_租赁负债',
     'B_长期应付款(合计)']:
        now_df['公司债务'] += stock_df[factor2]
    
    now_df['债务资本成本总额'] = stock_df['R_财务费用'] + stock_df['R_汇兑收益']
    now_df['经营资产自由现金流'] = stock_df['C_经营活动产生的现金流量净额'] - stock_df['C_固定资产折旧、油气资产折耗、生产性物资折旧'] - stock_df['C_无形资产摊销'] - stock_df['C_长期待摊费用摊销'] - stock_df['C_处置固定资产、无形资产和其他长期资产的损失']
    now_df['实际企业所得税税率'] = 1 - ((stock_df['R_四、利润总额'] - stock_df['R_减：所得税费用']) /  stock_df['R_四、利润总额'])
    now_df['少数股东权益比例'] = stock_df['B_少数股东权益'] / stock_df['B_所有者权益(或股东权益)合计']
    now_df['债务占比'] = now_df['公司债务'] / (stock_df['B_所有者权益(或股东权益)合计'] + now_df['公司债务'])
    now_df.drop_duplicates(subset=['财报日期'], inplace=True)
    now_df.reset_index(inplace=True)
    del now_df['index']

    print(now_df.tail(10))

    return now_df

def cal_WACC(now_df, time):
    WACC = (now_df['债务资本成本总额'] / ((now_df['公司债务'] + now_df['公司债务'].shift(time)) / 2) * now_df['债务占比'] * (1-now_df['实际企业所得税税率'])) + (0.09 * (1-now_df['债务占比']))
    return WACC.tolist()[-time]

def fcf_discounted(now_df, WACC, time, zero_change, one_change, two_change, three_change, g1, g2, g3, t1, t2):
    value = (now_df.loc[: ,'金融资产'].tolist()[-time] - now_df.loc[: ,'公司债务'].tolist()[-time])
    if zero_change == True:
        FCF = now_df.loc[: ,'经营资产自由现金流'].tolist()[-time] / WACC
    if one_change == True:
        FCF = (now_df.loc[: ,'经营资产自由现金流'].tolist()[-time] * (1+g1)) / (WACC - g1)
    if two_change == True: 
        temp_sum = 0
        for _ in t1:
            temp = now_df.loc[: ,'经营资产自由现金流'].tolist()[-time] * ((1+g1) ** _) / ((1+WACC) ** _)
            temp_sum = temp + temp_sum
        FCF = ((now_df.loc[: ,'经营资产自由现金流'].tolist()[-time] * ((1+g1) ** (t1[-1]-1)) * (1+g2)) / ((WACC-g2)*((1+WACC)**t1[-1]))) + temp_sum
    if three_change == True:
        temp_sum1, temp_sum2 = 0, 0
        for _ in t1:
            temp1 = now_df.loc[: ,'经营资产自由现金流'].tolist()[-time] * ((1+g1) ** _)
            temp =  temp1 / ((1+WACC) ** _)
            temp_sum1 = temp + temp_sum1
        for _ in t2:
            temp = temp1 * ((1+g2) ** _) / ((1+WACC) ** (_+t1[-1]))
            temp_sum2 = temp + temp_sum2
        FCF = (temp1 * ((1+g2) ** t2) * (1+g3)) / ((WACC-g3)*((1+WACC)**(t1[-1]+t2[-1]))) + temp_sum1 + temp_sum2
    FCF_plus_value =  (FCF + value)  * (1 - now_df.loc[: ,'少数股东权益比例'].tolist()[-time])
    result = FCF_plus_value / (now_df.loc[: ,'总市值'].tolist()[-time] / now_df.loc[: ,'收盘价'].tolist()[-time])  # 股票内在价值，价值/股数
    print('归属于上市公司股东的价值:', FCF_plus_value, '\n', '股票内在价值：', result)
    return FCF_plus_value, result


def statistics(now_df, time):
    PE1 = now_df.loc[: ,'总市值'].tolist()[-time] / now_df.loc[: ,'净利润TTM'].tolist()[-time]  # 市盈率
    PE2 = FCF_plus_value / now_df.loc[: ,'净利润TTM'].tolist()[-time]
    print('原PE: ', PE1, '估值PE：', PE2)
    for time_n in [1, 2, 3, time, time+1, time+2, time+3]:
        print('前%s个季度股票价格' % (time_n-1), now_df.loc[: ,'收盘价'].tolist()[-time_n])  # 股票收盘价


### 主程序

用来计算股票内在价值

In [45]:
#=== main
df, finance_df = read_file(file_name)
stock_df = merge_data(df, finance_df)
now_df = pd.DataFrame()
now_df = data_been_prepared(now_df, stock_df)
WACC = cal_WACC(now_df, time)
print('=============================')
print('WACC is ', WACC)
FCF_plus_value, result = fcf_discounted(now_df, WACC, time, zero_change, one_change, two_change, three_change, g1, g2, g3, t1, t2)
statistics(now_df, time)

          股票代码  股票名称       交易日期           总市值        净利润TTM    收盘价
4850  sz000977  浪潮信息 2020-12-09  4.132930e+10  1.039378e+09  28.43
4851  sz000977  浪潮信息 2020-12-10  4.084957e+10  1.039378e+09  28.10
4852  sz000977  浪潮信息 2020-12-11  4.051521e+10  1.039378e+09  27.87
4853  sz000977  浪潮信息 2020-12-14  4.041345e+10  1.039378e+09  27.80
4854  sz000977  浪潮信息 2020-12-15  4.118392e+10  1.039378e+09  28.33
Index(['股票代码', '股票名称', '交易日期', '总市值', '净利润TTM', '收盘价', '财报发布日期', '财报日期',
       'B_货币资金', 'B_交易性金融资产', 'B_衍生金融资产', 'B_应收票据及应收账款', 'B_应收票据', 'B_应收账款',
       'B_应收款项融资', 'B_应收利息', 'B_应收股利', 'B_其他应收款', 'B_买入返售金融资产', 'B_发放贷款及垫款',
       'B_可供出售金融资产', 'B_持有至到期投资', 'B_长期应收款', 'B_长期股权投资', 'B_投资性房地产',
       'B_所有者权益(或股东权益)合计', 'C_经营活动产生的现金流量净额', 'B_短期借款', 'B_交易性金融负债', 'B_应付利息',
       'B_应付短期债券', 'B_一年内到期的非流动负债', 'B_长期借款', 'B_应付债券', 'B_租赁负债',
       'B_长期应付款(合计)', 'R_财务费用', 'R_汇兑收益', 'R_四、利润总额', 'R_减：所得税费用',
       'C_固定资产折旧、油气资产折耗、生产性物资折旧', 'C_无形资产摊销', 'C_长期待摊费用摊销',
       'C_处置固定资产、无形资产和其他长期资产的损

### 循环测试
以下是进行g变量等情况的测试

In [52]:
a = []
for b in np.arange(0.01, 0.1, 0.01):

#=== main
    g1 = b
    df, finance_df = read_file(file_name)
    stock_df = merge_data(df, finance_df)
    now_df = pd.DataFrame()
    now_df = data_been_prepared(now_df, stock_df)
    print('=============================')
    FCF_plus_value, result = fcf_discounted(now_df, WACC, time, zero_change, one_change, two_change, three_change, g1, g2, g3)
    statistics(now_df, time)
    a.append(result)
    a.append(b)
print(a)

          股票代码  股票名称       交易日期           总市值        净利润TTM    收盘价
4850  sz000977  浪潮信息 2020-12-09  4.132930e+10  1.039378e+09  28.43
4851  sz000977  浪潮信息 2020-12-10  4.084957e+10  1.039378e+09  28.10
4852  sz000977  浪潮信息 2020-12-11  4.051521e+10  1.039378e+09  27.87
4853  sz000977  浪潮信息 2020-12-14  4.041345e+10  1.039378e+09  27.80
4854  sz000977  浪潮信息 2020-12-15  4.118392e+10  1.039378e+09  28.33
Index(['股票代码', '股票名称', '交易日期', '总市值', '净利润TTM', '收盘价', '财报发布日期', '财报日期',
       'B_货币资金', 'B_交易性金融资产', 'B_衍生金融资产', 'B_应收票据及应收账款', 'B_应收票据', 'B_应收账款',
       'B_应收款项融资', 'B_应收利息', 'B_应收股利', 'B_其他应收款', 'B_买入返售金融资产', 'B_发放贷款及垫款',
       'B_可供出售金融资产', 'B_持有至到期投资', 'B_长期应收款', 'B_长期股权投资', 'B_投资性房地产',
       'B_所有者权益(或股东权益)合计', 'C_经营活动产生现金流量净额', 'B_短期借款', 'B_交易性金融负债', 'B_应付利息',
       'B_应付短期债券', 'B_一年内到期的非流动负债', 'B_长期借款', 'B_应付债券', 'B_租赁负债',
       'B_长期应付款(合计)', 'C_固定资产折旧、油气资产折耗、生产性物资折旧', 'C_无形资产摊销', 'C_长期待摊费用摊销',
       'C_处置固定资产、无形资产和其他长期资产的损失', 'B_少数股东权益'],
      dtype='object')
        股票代码  

          股票代码  股票名称       交易日期           总市值        净利润TTM    收盘价
4850  sz000977  浪潮信息 2020-12-09  4.132930e+10  1.039378e+09  28.43
4851  sz000977  浪潮信息 2020-12-10  4.084957e+10  1.039378e+09  28.10
4852  sz000977  浪潮信息 2020-12-11  4.051521e+10  1.039378e+09  27.87
4853  sz000977  浪潮信息 2020-12-14  4.041345e+10  1.039378e+09  27.80
4854  sz000977  浪潮信息 2020-12-15  4.118392e+10  1.039378e+09  28.33
Index(['股票代码', '股票名称', '交易日期', '总市值', '净利润TTM', '收盘价', '财报发布日期', '财报日期',
       'B_货币资金', 'B_交易性金融资产', 'B_衍生金融资产', 'B_应收票据及应收账款', 'B_应收票据', 'B_应收账款',
       'B_应收款项融资', 'B_应收利息', 'B_应收股利', 'B_其他应收款', 'B_买入返售金融资产', 'B_发放贷款及垫款',
       'B_可供出售金融资产', 'B_持有至到期投资', 'B_长期应收款', 'B_长期股权投资', 'B_投资性房地产',
       'B_所有者权益(或股东权益)合计', 'C_经营活动产生现金流量净额', 'B_短期借款', 'B_交易性金融负债', 'B_应付利息',
       'B_应付短期债券', 'B_一年内到期的非流动负债', 'B_长期借款', 'B_应付债券', 'B_租赁负债',
       'B_长期应付款(合计)', 'C_固定资产折旧、油气资产折耗、生产性物资折旧', 'C_无形资产摊销', 'C_长期待摊费用摊销',
       'C_处置固定资产、无形资产和其他长期资产的损失', 'B_少数股东权益'],
      dtype='object')
        股票代码  

          股票代码  股票名称       交易日期           总市值        净利润TTM    收盘价
4850  sz000977  浪潮信息 2020-12-09  4.132930e+10  1.039378e+09  28.43
4851  sz000977  浪潮信息 2020-12-10  4.084957e+10  1.039378e+09  28.10
4852  sz000977  浪潮信息 2020-12-11  4.051521e+10  1.039378e+09  27.87
4853  sz000977  浪潮信息 2020-12-14  4.041345e+10  1.039378e+09  27.80
4854  sz000977  浪潮信息 2020-12-15  4.118392e+10  1.039378e+09  28.33
Index(['股票代码', '股票名称', '交易日期', '总市值', '净利润TTM', '收盘价', '财报发布日期', '财报日期',
       'B_货币资金', 'B_交易性金融资产', 'B_衍生金融资产', 'B_应收票据及应收账款', 'B_应收票据', 'B_应收账款',
       'B_应收款项融资', 'B_应收利息', 'B_应收股利', 'B_其他应收款', 'B_买入返售金融资产', 'B_发放贷款及垫款',
       'B_可供出售金融资产', 'B_持有至到期投资', 'B_长期应收款', 'B_长期股权投资', 'B_投资性房地产',
       'B_所有者权益(或股东权益)合计', 'C_经营活动产生现金流量净额', 'B_短期借款', 'B_交易性金融负债', 'B_应付利息',
       'B_应付短期债券', 'B_一年内到期的非流动负债', 'B_长期借款', 'B_应付债券', 'B_租赁负债',
       'B_长期应付款(合计)', 'C_固定资产折旧、油气资产折耗、生产性物资折旧', 'C_无形资产摊销', 'C_长期待摊费用摊销',
       'C_处置固定资产、无形资产和其他长期资产的损失', 'B_少数股东权益'],
      dtype='object')
        股票代码  

In [5]:
# pd.set_option('display.max_columns', 100)
finance_df = pd.read_csv(r'C:\Users\xueli\python_file\stock_quant\sina_financial_data\%s.csv' % file_name, parse_dates=['财报日期', '财报发布日期'], skiprows=1, encoding='gbk')
finance_df.columns.values.tolist()

['股票代码',
 '财报日期',
 '财报年份',
 '财报季度',
 '财报发布日期',
 'R_一、营业总收入',
 'R_营业收入',
 'R_二、营业总成本',
 'R_营业成本',
 'R_营业税金及附加',
 'R_销售费用',
 'R_管理费用',
 'R_财务费用',
 'R_研发费用',
 'R_资产减值损失',
 'R_公允价值变动收益',
 'R_投资收益',
 'R_其中:对联营企业和合营企业的投资收益',
 'R_汇兑收益',
 'R_三、营业利润',
 'R_加:营业外收入',
 'R_减：营业外支出',
 'R_其中：非流动资产处置损失',
 'R_四、利润总额',
 'R_减：所得税费用',
 'R_五、净利润',
 'R_归属于母公司所有者的净利润',
 'R_少数股东损益',
 'R_六、每股收益',
 'R_基本每股收益(元/股)',
 'R_稀释每股收益(元/股)',
 'R_七、其他综合收益',
 'R_八、综合收益总额',
 'R_归属于母公司所有者的综合收益总额',
 'R_归属于少数股东的综合收益总额',
 'C_一、经营活动产生的现金流量',
 'C_销售商品、提供劳务收到的现金',
 'C_收到的税费返还',
 'C_收到的其他与经营活动有关的现金',
 'C_经营活动现金流入小计',
 'C_购买商品、接受劳务支付的现金',
 'C_支付给职工以及为职工支付的现金',
 'C_支付的各项税费',
 'C_支付的其他与经营活动有关的现金',
 'C_经营活动现金流出小计',
 'C_经营活动产生的现金流量净额',
 'C_二、投资活动产生的现金流量',
 'C_收回投资所收到的现金',
 'C_取得投资收益所收到的现金',
 'C_处置固定资产、无形资产和其他长期资产所收回的现金净额',
 'C_处置子公司及其他营业单位收到的现金净额',
 'C_收到的其他与投资活动有关的现金',
 'C_投资活动现金流入小计',
 'C_购建固定资产、无形资产和其他长期资产所支付的现金',
 'C_投资所支付的现金',
 'C_取得子公司及其他营业单位支付的现金净额',
 'C_支付的其他与投资活动有关的现金',
 'C_投资活动现金流出小计',
 'C_投资活动产生的现金流量净额',
 'C_三、筹资活动产生的现金流量',
